In [1]:
import torch
import matplotlib.pyplot as plt
from trainSettings import *
from helpersPlot import *
import pandas as pd


In [4]:
trajs, labels = models_phenom().single_state(1, 
                            L=0,
                            T=T,
                            Ds=0.1,  # Mean and variance
                            alphas=1)
# Reshape trajectories
trajs = trajs.transpose(1, 0, 2)
trajs = trajs / traj_div_factor


vids = trajectories_to_video(trajs,nPosPerFrame,False,image_props)

vids = trajectories_to_video(np.zeros_like(trajs),nPosPerFrame,False,image_props)
play_video(vids[0], vmin=0, vmax=7500)
trajs, labels = models_phenom().single_state(1, 
                            L=0,
                            T=T,
                            Ds=7,  # Mean and variance
                            alphas=1)
# Reshape trajectories
trajs = trajs.transpose(1, 0, 2)
trajs = trajs / traj_div_factor
vids = trajectories_to_video(trajs,nPosPerFrame,False,image_props)
play_video(vids[0], vmin=0, vmax=7500)

vmin: 0 vmax: 7500 mean: 1840.48


vmin: 0 vmax: 7500 mean: 1841.05


In [3]:
# Load validation dataset (fixed, does not change across cycles)
val_videos = load_validation_data(length=nFrames)  # Returns (vid1, vid3, vid5, vid7, vid_in_order)
val_vid_in_order = val_videos[4]
print(val_d_in_order.shape)
val_labels = torch.tensor([1, 3, 5, 7], dtype=torch.float32)  # Corresponding labels

(70,)
